<a href="https://colab.research.google.com/github/Tim-Albiges/Tim-Albiges/blob/main/Langchain_Piping_serialised_and_parallel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install and load Libraries

In [6]:
# install libraries, require restart of kernel.
from IPython.display import clear_output
!pip install langchain-community
!pip install langchain_core
!pip install ollama
clear_output()

In [7]:
from langchain.prompts import PromptTemplate
from langchain_community.llms import Ollama
from langchain_core.output_parsers import StrOutputParser

#Download and run the server within Colab's Notebook

In [8]:
#Install Ollama server with shell
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [14]:
#Run server
!nohup ollama serve > ollama.log 2>&1 &

In [15]:
#Pull the llama3 model in the server
!ollama pull llama3

#Sequentially

In [16]:
# 1. Define the components
prompt = PromptTemplate.from_template("Tell me a short joke about {topic}")
model = Ollama(model="llama3")
parser = StrOutputParser()

In [17]:
# 2. Chain them sequentially using the pipe operator
joke_chain = prompt | model | parser

In [19]:
# 3. Invoke the chain with an input
response = joke_chain.invoke({"topic": "AI"})
print(response)


Why did the AI program go on a diet?

Because it wanted to lose some bytes! (get it? "bytes" instead of "pounds"? Ah, I'll just shut down now)


#Parallel


In [20]:
#load the parallel library
from langchain_core.runnables import RunnableParallel

In [21]:
# 1. Define the components (same as before)
model = Ollama(model="llama3")
parser = StrOutputParser()

In [22]:
# 2. Define two independent chains
joke_chain = PromptTemplate.from_template("Tell me a short joke about {topic}") | model | parser
poem_chain = PromptTemplate.from_template("Write a 2-line poem about {topic}") | model | parser

In [23]:
# 3. Combine them in parallel using a dictionary
final_chain = RunnableParallel({
    "joke_output": joke_chain,
    "poem_output": poem_chain
})

In [25]:
# 4. Invoke the chain with a single input. Both tasks run concurrently.
response = final_chain.invoke({"topic": "bears"})
print(response)

{'joke_output': 'Why did the bear go to the doctor?\n\nBecause it had a grizzly cough!', 'poem_output': 'Here is a 2-line poem about bears:\n\nStrong and fierce, they roam the night,\nGuarding their dens with gentle might.'}
